In [185]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr

import rioxarray

In [186]:
data_dir = Path('../data/outputs')
sites_dir = data_dir / "sites"
sites_metrics_dir = sites_dir / "metrics" / "x1-y1-z1"
sites_netcdf_dir = sites_metrics_dir / "net_cdf"
sites_grid_tif_dir = sites_metrics_dir / "grid_tif"
sites_grid_tif_dir.mkdir(parents=True, exist_ok=True)
sites_voxel_tif_dir = sites_metrics_dir / "voxel_tif"
sites_voxel_tif_dir.mkdir(parents=True, exist_ok=True)


plots_dir = data_dir / "plots"
plot_metrics_dir = plots_dir / "metrics" / "x1-y1-z1"
plot_metrics_dir.mkdir(parents=True, exist_ok=True)
plot_net_cdf_dir = plot_metrics_dir / "net_cdf"
plot_net_cdf_dir.mkdir(parents=True, exist_ok=True)
plot_grid_tif_dir = plot_metrics_dir / "grid_tif"
plot_grid_tif_dir.mkdir(parents=True, exist_ok=True)
plot_voxel_tif_dir = plot_metrics_dir / "voxel_tif"
plot_voxel_tif_dir.mkdir(parents=True, exist_ok=True)

In [187]:
sites_gdf = gpd.read_file(sites_dir / "sites.geojson")
site_ids = sites_gdf['site'].to_list()

In [188]:
plots_gdf = gpd.read_file(plots_dir / "plots.geojson")
plot_site_geom_tuples = list(
    zip(
        plots_gdf["site_plot_id"].astype(str),
        plots_gdf["site"].astype(str),
        plots_gdf["geometry"],
    )
)

In [ ]:
def read_site_metrics(site_id: str) -> xr.Dataset:
    metrics_path = sites_netcdf_dir / f"{site_id}.nc"
    ds = xr.open_dataset(metrics_path, decode_coords="all")
    return ds

def clip_metrics(ds: xr.Dataset, geom) -> xr.Dataset:
    clipped = ds.rio.clip([geom])
    return clipped

def save_grid_metrics(metrics: xr.Dataset, id: str, output_dir: Path):
    output_path = output_dir / f"{id}.tif"
    grid_metric_keys = [k for k in metrics.data_vars if metrics[k].ndim == 2]
    grid_metrics = metrics[grid_metric_keys]
    
    grid_metrics.rio.to_raster(output_path)

def flatten_voxel_metrics(metrics: xr.Dataset) -> list[xr.Dataset]:
    voxel_metric_keys = [k for k in metrics.data_vars if metrics[k].ndim == 3]
    voxel_metrics = metrics[voxel_metric_keys]
    
    flatted_datasets = []

    for metric in voxel_metrics.data_vars:
        # Select the variable as a new dataset
        ds_var = voxel_metrics[[metric]]
        
        # Create a new dataset with one variable per z slice
        new_vars = {}
        for i, z_val in enumerate(ds_var['z'].values):
            # Select the 2D slice at this z
            data_2d = ds_var[metric].sel(z=z_val)
            # Name for the new variable
            new_var_name = f"z{i}"
            # Add to dict
            new_vars[new_var_name] = (('y', 'x'), data_2d.values)
        
        # Build the new dataset
        ds_split = xr.Dataset(new_vars, coords={'y': ds_var['y'], 'x': ds_var['x']})
        ds_split.attrs = voxel_metrics.attrs
        ds_split.attrs['metric'] = metric
        
        flatted_datasets.append(ds_split)

    return flatted_datasets

def save_voxel_metrics(metrics: xr.Dataset, id: str, output_dir: Path):
    flatted_datasets = flatten_voxel_metrics(metrics)
    for ds in flatted_datasets:
        metric = ds.attrs['metric']
        output_path = output_dir / f"{id}_{metric}.tif"
        ds.rio.to_raster(output_path)

def read_and_clip_plot_metrics(plot_id: str, site_id: str, geom) -> xr.Dataset:
    ds = read_site_metrics(site_id)
    clipped = clip_metrics(ds, geom)
    clipped.attrs['site_plot_id'] = plot_id
    clipped = clipped.transpose('y', 'x', 'z')

    z_max = clipped['max_height'].max().item()
    clipped = clipped.sel(z=slice(0, np.ceil(z_max)))

    clipped.to_netcdf(plot_net_cdf_dir / f"{plot_id}.nc")
    save_grid_metrics(clipped, plot_id, plot_grid_tif_dir)
    save_voxel_metrics(clipped, plot_id, plot_voxel_tif_dir)
    return clipped

def read_and_save_site_metrics(site_id: str):
    ds = read_site_metrics(site_id)
    ds = ds.transpose('y', 'x', 'z')
    ds.load()
    ds.close()
    # ds.to_netcdf(sites_netcdf_dir / f"{site_id}.nc")
    save_grid_metrics(ds, site_id, sites_grid_tif_dir)
    save_voxel_metrics(ds, site_id, sites_voxel_tif_dir)
    return ds

In [ ]:
plot = read_and_clip_plot_metrics(*plot_site_geom_tuples[1])
plot

In [ ]:
read_and_save_site_metrics(site_ids[0])

In [ ]:
for site in site_ids:
    print(f"Processing site {site}")
    read_and_save_site_metrics(site)

In [ ]:
for p in plot_site_geom_tuples:
    print(f"Processing plot {p[0]} from site {p[1]}")
    read_and_clip_plot_metrics(*p)

# Plot Height Profiles

For each plot, mean voxels in the x and y dimensions to get some height profiles.

In [189]:
plot_profile_dir = plot_metrics_dir / "voxel_profile_json"
plot_profile_dir.mkdir(parents=True, exist_ok=True)

In [191]:
plot_ids = list(plots_gdf["site_plot_id"])

In [199]:
def create_height_profile(plot_id: str):
    metrics_path = plot_net_cdf_dir / f"{plot_id}.nc"
    ds = xr.open_dataset(metrics_path, decode_coords="all")
    voxel_metric_keys = [k for k in ds.data_vars if ds[k].ndim == 3]
    voxel_metrics = ds[voxel_metric_keys]
    height_profile = voxel_metrics.mean(dim=['x', 'y']).to_dataframe()
    height_profile.reset_index(inplace=True)
    height_profile['site_plot_id'] = plot_id
    output_path = plot_profile_dir / f"{plot_id}.json"
    height_profile.to_json(output_path, orient='records')
    return height_profile

In [200]:
all_profiles = pd.concat([create_height_profile(pid) for pid in plot_ids])

In [201]:
all_profiles.to_json(plot_profile_dir / "all_profiles.json", orient='records')